<a href="https://colab.research.google.com/github/tetherless-world/CodeGraph/blob/master/embeddingsTest/docstring_USE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl


--2020-05-26 18:10:43--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200526T181043Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEE4aCXVzLWVhc3QtMSJGMEQCIBKC2aQ%2Bh2%2B5n%2FJ6bgiNJQ%2BPZhrTdSzc42OrG8tVQjKTAiAsQnGAsVEyS5R2O8%2FJWqhTgk0PGCSKki5WY6%2BPvEyf%2Biq9Awin%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAAaDDQ1NTg2N

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle
import tensorflow as tf
data_file = open('drive/My Drive/self services/MS/RPI/TWC/docStringPickled_class', 'rb') 
docStringCompiled= pickle.load(data_file) 
data_file.close()
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


def embed(input):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
    model = hub.load(module_url)
    return model(input)
def randomize(x, y):
    ##Randomizes the order of data samples and their corresponding labels
    import numpy as np
    permutation = np.random.permutation(len(y))
    y=np.asarray(y)
    shuffled_x = x[permutation]
    shuffled_y = y[permutation]

    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    ##get the specified batch
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch
import faiss                   # make faiss available

size_to_train=13000 ##ive run for a max of 130000 till now
batch_size=100
embeded_docnames=[]
embeded_docmessages=[]
embeded_stringdocmessages=[]
logging.set_verbosity(logging.ERROR)
index = faiss.IndexFlatL2(512)
shuffled_docnames,shuffled_docmessages=randomize(np.asarray([seq[0] for seq in docStringCompiled[:3*size_to_train]]),np.asarray([seq[1] for seq in docStringCompiled[:size_to_train]]))


import faiss                   # make faiss available

size_to_train=6500 ##ive run for a max of 130000 till now
batch_size=100
embeded_docnames=[]
embeded_docmessages=[]
embeded_stringdocmessages=[]

index = faiss.IndexFlatL2(512)
shuffled_docnames,shuffled_docmessages=randomize(np.asarray([seq[0] for seq in docStringCompiled[2*size_to_train:5*size_to_train]]),np.asarray([seq[1] for seq in docStringCompiled[2*size_to_train:5*size_to_train]]))

for i in range(int(size_to_train/batch_size)):
    print("iteration",i)
    ###use stream prtocol, only index in memory
# Reduce logging output.
    docStringsset1,docStringsset2=get_next_batch(shuffled_docnames,shuffled_docmessages,i*batch_size,(i+1)*batch_size)
    print(len(docStringsset2))
    message_embeddings = embed(docStringsset2)
    message_embeddings=tf.make_tensor_proto(message_embeddings)
    message_embeddings=tf.make_ndarray(message_embeddings)
    embeded_docnames.append(docStringsset1)
    embeded_docmessages.append(message_embeddings)
    embeded_stringdocmessages.append(docStringsset2)
    index.add(message_embeddings)

k=6
embeded_distance_index_info=[]
embeded_distance_info=[]
for i in range(len(embeded_docmessages)):
# we want to see 6 nearest neighborS
    D, I = index.search(embeded_docmessages[i], k) 
    embeded_distance_index_info.append(I)
    embeded_distance_info.append(D)
import sys
sys.stdout = open("output_embedding_docstrings_clean_out_CLASS_2.txt", "w")

for i in range(len(embeded_docmessages)):
    for j in range(len(embeded_docmessages[i])):
        print("-------------------------------------------------------------")
        print("document name  : \n"+str(embeded_docnames[i][j])+"\n")
        for k in range(len(embeded_distance_index_info[i][j])):
            print("\n close to document :",shuffled_docnames[embeded_distance_index_info[i][j][k]])
            print("\n with a distance :",embeded_distance_info[i][j][k])


sys.stdout.close()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


iteration 0
100
iteration 1
100
iteration 2
100
iteration 3
100
iteration 4
100
iteration 5
100
iteration 6
100
iteration 7
100
iteration 8
100
iteration 9
100
iteration 10
100
iteration 11
100
iteration 12
100
iteration 13
100
iteration 14
100
iteration 15
100
iteration 16
100
iteration 17
100
iteration 18
100
iteration 19
100
iteration 20
100
iteration 21
100
iteration 22
100
iteration 23
100
iteration 24
100
iteration 25
100
iteration 26
100
iteration 27
100
iteration 28
100
iteration 29
100
iteration 30
100
iteration 31
100
iteration 32
100
iteration 33
100
iteration 34
100
iteration 35
100
iteration 36
100
iteration 37
100
iteration 38
100
iteration 39
100
iteration 40
100
iteration 41
100
iteration 42
100
iteration 43
100
iteration 44
100
iteration 45
100
iteration 46
100
iteration 47
100
iteration 48
100
iteration 49
100
iteration 50
100
iteration 51
100
iteration 52
100
iteration 53
100
iteration 54
100
iteration 55
100
iteration 56
100
iteration 57
100
iteration 58
100
iterati

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 239, in dispatch_shell
    sys.stdout.flush()
ValueError: I/O operation on closed file.
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 456, in _handle_events
    self._handle_recv()
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 486, in _handle_recv
   

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 239, in dispatch_shell
    sys.stdout.flush()
ValueError: I/O operation on closed file.
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 456, in _handle_events
    self._handle_recv()
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 486, in _handle_recv
   

In [0]:
import pickle
import tensorflow as tf
data_file = open('drive/My Drive/self services/MS/RPI/TWC/docStringPickled_class', 'rb') 
docStringCompiled= pickle.load(data_file) 
data_file.close()
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


def embed(input):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
    model = hub.load(module_url)
    return model(input)
def randomize(x, y):
    ##Randomizes the order of data samples and their corresponding labels
    import numpy as np
    permutation = np.random.permutation(len(y))
    y=np.asarray(y)
    shuffled_x = x[permutation]
    shuffled_y = y[permutation]

    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    ##get the specified batch
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch
import faiss                   # make faiss available

size_to_train=13000 ##ive run for a max of 130000 till now
batch_size=100
embeded_docnames=[]
embeded_docmessages=[]
embeded_stringdocmessages=[]
logging.set_verbosity(logging.ERROR)
index = faiss.IndexFlatL2(512)
shuffled_docnames,shuffled_docmessages=randomize(np.asarray([seq[0] for seq in docStringCompiled[:3*size_to_train]]),np.asarray([seq[1] for seq in docStringCompiled[:size_to_train]]))


import faiss                   # make faiss available

size_to_train=6500 ##ive run for a max of 130000 till now
batch_size=100
embeded_docnames=[]
embeded_docmessages=[]
embeded_stringdocmessages=[]

index = faiss.IndexFlatL2(512)
shuffled_docnames,shuffled_docmessages=randomize(np.asarray([seq[0] for seq in docStringCompiled[3*size_to_train:4*size_to_train]]),np.asarray([seq[1] for seq in docStringCompiled[3*size_to_train:4*size_to_trai]]))

for i in range(int(size_to_train/batch_size)):
    print("iteration",i)
    
# Reduce logging output.
    docStringsset1,docStringsset2=get_next_batch(shuffled_docnames,shuffled_docmessages,i*batch_size,(i+1)*batch_size)
    print(len(docStringsset2))
    message_embeddings = embed(docStringsset2)
    message_embeddings=tf.make_tensor_proto(message_embeddings)
    message_embeddings=tf.make_ndarray(message_embeddings)
    embeded_docnames.append(docStringsset1)
    embeded_docmessages.append(message_embeddings)
    embeded_stringdocmessages.append(docStringsset2)
    index.add(message_embeddings)

k=6
embeded_distance_index_info=[]
embeded_distance_info=[]
for i in range(len(embeded_docmessages)):
# we want to see 6 nearest neighborS
    D, I = index.search(embeded_docmessages[i], k) 
    embeded_distance_index_info.append(I)
    embeded_distance_info.append(D)
import sys
sys.stdout = open("output_embedding_docstrings_clean_out_CLASS_2.txt", "w")

for i in range(len(embeded_docmessages)):
    for j in range(len(embeded_docmessages[i])):
        print("-------------------------------------------------------------")
        print("document name  : \n"+str(embeded_docnames[i][j])+"\n")
        for k in range(len(embeded_distance_index_info[i][j])):
            print("\n close to document :",shuffled_docnames[embeded_distance_index_info[i][j][k]])
            print("\n with a distance :",embeded_distance_info[i][j][k])


sys.stdout.close()

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 239, in dispatch_shell
    sys.stdout.flush()
ValueError: I/O operation on closed file.
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 456, in _handle_events
    self._handle_recv()
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 486, in _handle_recv
   